In [5]:
from crocoddyl import *
import pinocchio as pin
import numpy as np

robot = loadKintonArm()
#robot.initDisplay(loadModel=True)
#robot.display(robot.q0)

robot.framesForwardKinematics(robot.q0)

# Create a cost model per the running and terminal action model.
runningCostModel = CostModelSum(robot.model)
terminalCostModel = CostModelSum(robot.model)

In [6]:
target_pos  = np.array([-0.1,-0.05,-0.15])
target_quat = pin.Quaternion(.4, .02, -.5, .7)
target_quat.normalize()

# Plot goal frame
robot.viewer.gui.addXYZaxis('world/framegoal', [1., 0., 0., 1.], .015, 4)
robot.viewer.gui.applyConfiguration('world/framegoal', target_pos.tolist() + [target_quat[0], target_quat[1], target_quat[2], target_quat[3]])
robot.viewer.gui.refresh()

IDX_BASE = robot.model.getFrameId('base_link', pin.FrameType.BODY)
Mbase = robot.data.oMf[IDX_BASE]
robot.viewer.gui.addXYZaxis('world/framebase', [1., 0., 0., 1.], .015, 4)
robot.viewer.gui.applyConfiguration('world/framebase', pin.se3ToXYZQUATtuple(Mbase))
robot.viewer.gui.refresh()
Mbase

AttributeError: 'NoneType' object has no attribute 'viewer'

In [7]:
# Note that we need to include a cost model (i.e. set of cost functions) in
# order to fully define the action model for our optimal control problem.
# For this particular example, we formulate three running-cost functions:
# goal-tracking cost, state and control regularization; and one terminal-cost:
# goal cost. First, let's create the common cost functions
frameName = 'link6'
state = StatePinocchio(robot.model)
SE3ref = pin.SE3()
SE3ref.translation = target_pos.reshape(3,1)
SE3ref.rotation = target_quat.matrix()

goalTrackingCost = CostModelFramePlacement(robot.model, nu=robot.model.nv, frame=robot.model.getFrameId(frameName), ref=SE3ref)
xRegCost = CostModelState(robot.model, state, ref=state.zero(), nu=robot.model.nv)
uRegCost = CostModelControl(robot.model, nu=robot.model.nv)

# Then let's add the running and terminal cost functions
runningCostModel.addCost(name="pos", weight=1e-3, cost=goalTrackingCost)
runningCostModel.addCost(name="regx", weight=1e-7, cost=xRegCost)
runningCostModel.addCost(name="regu", weight=1e-7, cost=uRegCost)
terminalCostModel.addCost(name="pos", weight=50, cost=goalTrackingCost)

In [8]:
# Next, we need to create an action model for running and terminal knots. The
# forward dynamics (computed using ABA) are implemented
# inside DifferentialActionModelFullyActuated.
runningModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, runningCostModel))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, terminalCostModel))

# Defining the time duration for running action models and the terminal one
dt = 1e-3
runningModel.timeStep = dt

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 250
q0 = [0., 0., 0., 0., 0., 0.]
x0 = np.hstack([q0, np.zeros(robot.model.nv)])
problem = ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
# ddp.callback = [CallbackDDPVerbose()]
# ddp.callback.append(CallbackDDPLogger())

# Solving it with the DDP algorithm
ddp.solve()


([array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([-8.56312177e-04,  1.89532152e-03,  2.73421184e-03,  1.40535055e-04,
         -6.80910601e-03, -1.47306350e-02, -8.56312177e-01,  1.89532152e+00,
          2.73421184e+00,  1.40535055e-01, -6.80910601e+00, -1.47306350e+01]),
  array([-2.54926553e-03,  5.12874409e-03,  7.81062478e-03, -3.01186931e-05,
         -1.90322663e-02, -4.14852078e-02, -1.69295335e+00,  3.23342257e+00,
          5.07641293e+00, -1.70653748e-01, -1.22231603e+01, -2.67545728e+01]),
  array([-5.08231018e-03,  9.22720244e-03,  1.48413991e-02, -8.04175234e-04,
         -3.54988967e-02, -7.79179637e-02, -2.53304466e+00,  4.09845835e+00,
          7.03077430e+00, -7.74056541e-01, -1.64666304e+01, -3.64327559e+01]),
  array([-8.46800128e-03,  1.37903845e-02,  2.34569133e-02, -2.35975061e-03,
         -5.52243122e-02, -1.22000964e-01, -3.38569110e+00,  4.56318206e+00,
          8.61551424e+00, -1.55557537e+00, -1.97254155e+01, -4.40830008e+01]),
  array([

In [9]:
from crocoddyl.diagnostic import displayTrajectory

displayTrajectory(robot, ddp.xs, runningModel.timeStep)

/home/jmarti/robotics/toolboxes/crocoddyl/crocoddyl/diagnostic.py:75: DeprecatedWarning: Call to deprecated function initDisplay. Use initViewer
  robot.initDisplay(loadModel=True)
